## Converting MIP mps files to LP files

#### For larger problem sizes, we only save the matrix outputs to then load into the polyhedral model code

In [1]:
import gurobipy as gp
import numpy as np
import scipy.sparse as sp
import os

In [2]:
def convert_mps_to_continuous(input_mps, output_mps):
    try:
        # Load the MPS file
        model = gp.read(input_mps)
        
        # Relax the model (convert integer and binary variables to continuous)
        model = model.relax()

        # Write the relaxed model to a new MPS file
        output = os.path.join('{}.mps'.format(output_mps))
        model.write(output)
        os.rename(output, output_mps)
        print(f"Converted MPS file saved as {output_mps}")
        
    except gp.GurobiError as e:
        print(f"Error: {e}")

In [3]:
def large_prob_convert(input_mps, output_mps, mip):
    has_A = True
    has_B = True

    model = gp.read(input_mps)

    for var in model.getVars():
        lb = var.LB  # Lower bound
        ub = var.UB  # Upper bound
        
        # Add constraint for lower bound: x >= LB(x)
        if lb is not None:
            model.addConstr(var >= lb, name=f"lb_{var.VarName}")
        
        # Add constraint for upper bound: x <= UB(x)
        if ub is not None:
            model.addConstr(var <= ub, name=f"ub_{var.VarName}")

    model.update()

    # Relax the model (convert integer and binary variables to continuous)
    if mip:
        model = model.relax()

    # Extract sparse constraint matrix (A) and RHS (b) for all constraints
    con_mat = model.getA()  # Keep sparse format
    rhs = np.array([constr.RHS for constr in model.getConstrs()], dtype=float)  # Right-hand side vector

    # Get the sense of constraints (<=, >=, =)
    senses = [constr.Sense for constr in model.getConstrs()]

    # Initialize sparse lists for constraint types
    eq_rows, leq_rows, geq_rows = [], [], []
    b_eq, b_leq, b_geq = [], [], []

    # Separate constraints by type
    for i, s in enumerate(senses):
        if s == '=':
            eq_rows.append(con_mat.getrow(i))
            b_eq.append(rhs[i])
        elif s == '<':
            leq_rows.append(con_mat.getrow(i))
            b_leq.append(rhs[i])
        elif s == '>':
            geq_rows.append(con_mat.getrow(i))
            b_geq.append(rhs[i])

    if len(eq_rows) == 0:
        has_A = False
    # Stack sparse matrices and convert RHS vectors
    A = sp.vstack(eq_rows) if eq_rows else sp.csr_matrix((0, con_mat.shape[1]))
    B_leq = sp.vstack(leq_rows) if leq_rows else sp.csr_matrix((0, con_mat.shape[1]))
    B_geq = sp.vstack(geq_rows) if geq_rows else sp.csr_matrix((0, con_mat.shape[1]))

    b = np.array(b_eq, dtype=float) if b_eq else np.zeros(0)
    d_leq = np.array(b_leq, dtype=float) if b_leq else np.zeros(0)
    d_geq = np.array(b_geq, dtype=float) if b_geq else np.zeros(0)

    # Adjust signs for greater-than-or-equal constraints
    B_geq = -B_geq
    d_geq = -d_geq

    # Combine less-than-or-equal and greater-than-or-equal constraints
    if B_leq.shape[0] > 0 and B_geq.shape[0] > 0:
        B = sp.vstack([B_leq, B_geq])
        d = np.hstack([d_leq, d_geq])
    elif B_leq.shape[0] > 0:
        B = B_leq
        d = d_leq
    elif B_geq.shape[0] > 0:
        B = B_geq
        d = d_geq
    else:
        # B = sp.csr_matrix((0, con_mat.shape[1]))
        has_B = False

    # print(B)

    # Objective function coefficients (c)
    c = np.array([var.Obj for var in model.getVars()], dtype=float)
    
    if has_A:
        sp.save_npz(f"{output_mps}_A.npz", A)
        np.savetxt(f"{output_mps}_b.csv", b, delimiter=',', fmt='%.18e')
    if has_B:
        sp.save_npz(f"{output_mps}_B.npz", B)
        np.savetxt(f"{output_mps}_d.csv", d, delimiter=',', fmt='%.18e')
    if has_A or has_B: 
        np.savetxt(f"{output_mps}_c.csv", c, delimiter=',', fmt='%.18e')
    
    print(f"Matrices saved for file {output_mps}")


In [4]:
###MIPs reach INT MAX or do not give a useful error when try to read them
# problems = ['bab1', 'bab2', 'bab6', 'ds']

###MIPs that don't need to have matrices saved
# problems = [ '2club200v15p5scn', '8div-n59k10', '8div-n59k11','bg512142', 'bppc6-02','dc1c', 'dg012142', 'dolom1']

####MIPs with free-form MPS files that need to converted to fixed-form (saved matrices of problem instead)
problems = [
    'supportcase19', 'n3div36', 
    # 'academictimetablesmall', 'fiball', 'cryptanalysiskb128n5obj16', 'square47', 
    'tbfp-network', 'rail01', 's250r10','academictimetablebig','adult-max5features', 'allcolor10', 
    # 'amaze22012-07-04i', 
    'atm20-100', 'cmflsp40-24-10-7', 'comp16-3idx', 
    # 'cryptanalysiskb128n5obj14', 
    'dws008-03'
    # , 'elitserienhandball11i','elitserienhandball13i','elitserienhandball14i'
]

####Large Problems I was able to find (see email from Jon's contact at Gurobi for more details)
# problems = ['bharat', 'L2CTA3D', 'tpl-tub-ws1617', 'zib01']


problem_dir = 'miplib_subset'
gurobi_dir = 'gurobi_test_subset'
mps_prob_dir = 'mps_problems'
mat_prob_dir = 'matrix_problems'
if not os.path.exists(mps_prob_dir): os.mkdir(mps_prob_dir)
if not os.path.exists(mat_prob_dir): os.mkdir(mat_prob_dir)

for problem in problems:
    mps_in = os.path.join(problem_dir, problem)
    # mps_in = os.path.join(gurobi_dir, problem)
    
    ###only coverts MIPs to LPs
    # mps_out = os.path.join(mps_prob_dir, problem)
    # convert_mps_to_continuous(mps_in, mps_out)
    
    ###coverts MIPs to LPs and saved matrices
    mps_out = os.path.join(mat_prob_dir, problem)
    # large_prob_convert(mps_in,mps_out, True)
    large_prob_convert(mps_in,mps_out, True)

    

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01
Read MPS format model from file miplib_subset\supportcase19.mps
Reading time = 5.95 seconds
supportcase19: 10713 rows, 1429098 columns, 4287094 nonzeros
Matrices saved for file matrix_problems\supportcase19
Read MPS format model from file miplib_subset\n3div36.mps
Reading time = 0.91 seconds
n3div36: 4484 rows, 22120 columns, 340740 nonzeros
Matrices saved for file matrix_problems\n3div36
Read MPS format model from file miplib_subset\tbfp-network.mps
Reading time = 0.47 seconds
tbfp-network: 2436 rows, 72747 columns, 215837 nonzeros
Matrices saved for file matrix_problems\tbfp-network
Read MPS format model from file miplib_subset\rail01.mps
Reading time = 0.68 seconds
rail01: 46843 rows, 117527 columns, 392086 nonzeros
Matrices saved for file matrix_problems\rail01
Read MPS format model from file miplib_subset\s250r10.mps
Reading time = 3.29 seconds
s250r10: 10962 rows, 273142 columns, 1318607 no